In [1]:
import numpy as np
import pandas as pd
import numpy as np
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from IPython.display import display

In [2]:
dataset=[]
real_labels=[]
count=0

with open('sentiment-analysis.csv', newline='') as csvfile:
    
    reader = csv.reader(csvfile)
    
    for row in reader:
        if(count!=0):
          result=(str(row[0]).split(","))
          if(len(result)>1): 
            dataset.append(result[0])
            sentiment=result[1]
            if(sentiment[1] == 'P'):
                real_labels.append(1)
            else:
                real_labels.append(0)
            
            
                    
            
        count+=1  

In [3]:
# vetorização dos textos
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(dataset)


In [4]:
##################### Holdout ##################
# 30% do conjunto de dados é utilizado para os testes e 70% é utilizado para o treinamento
x_train, x_test, y_train, y_test = train_test_split(X, real_labels, test_size=0.3, random_state=42)

svc = SVC(kernel='linear')
svc.fit(x_train, y_train)

y_pred = svc.predict(x_test)

# avaliação do modelo
print(classification_report(y_test, y_pred))
conf_matrix = confusion_matrix(y_test, y_pred)

labels_name = ['Negativo', 'Positivo']

# Criar um DataFrame com 16 linhas e duas colunas
data = {'Verdadeiro': ['Positivo' if i==1 else 'Negativo' for i in y_test], 'Previsto': ['Positivo' if i==1 else 'Negativo' for i in y_pred]}
df = pd.DataFrame(data)
display(df)
        


              precision    recall  f1-score   support

           0       0.85      1.00      0.92        11
           1       1.00      0.89      0.94        18

    accuracy                           0.93        29
   macro avg       0.92      0.94      0.93        29
weighted avg       0.94      0.93      0.93        29



,Verdadeiro,Previsto
0,Negativo,Negativo
1,Positivo,Positivo
2,Positivo,Positivo
3,Negativo,Negativo
4,Positivo,Positivo
5,Positivo,Negativo
6,Positivo,Positivo
7,Positivo,Positivo
8,Positivo,Negativo
9,Positivo,Positivo


In [5]:
############# Validação Cruzada ############################

num_folds = 5
fold_size = X.shape[0] // num_folds

svc = SVC(kernel='linear')

 # Conversão da matriz esparsa X para uma matriz densa. 
X_dense = X.toarray()

In [6]:
count_fold=1
for i in range(num_folds):
    # Dividindo os conjuntos para cada fold
    inicio = i * fold_size
    fim = (i + 1) * fold_size
    data_train=[]
    data_test=[]
    labels_train=[]
    labels_test=[]


    for j in range(0, inicio):
       data_train.append(X_dense[j])
       labels_train.append(real_labels[j])
    
    for j in range(fim, X.shape[0]):
       data_train.append(X_dense[j])
       labels_train.append(real_labels[j])

    for j in range(inicio,fim):
       data_test.append(X_dense[j])
       labels_test.append(real_labels[j])   
    
    svc.fit(data_train, labels_train)

    # Avaliação modelo
    y_pred = svc.predict(data_test)
    acc_score= accuracy_score(labels_test, y_pred)
    print(f'**********Fold {count_fold}****************')
    print(acc_score)
    print(labels_test)
    print(y_pred )
    count_fold+=1

**********Fold 1****************
1.0
[1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
[1 0 1 0 1 0 1 1 1 0 1 0 1 0 1 0 1 0 1]
**********Fold 2****************
0.8421052631578947
[0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0]
[0 1 0 0 0 1 1 1 1 0 1 0 1 0 1 0 1 1 0]
**********Fold 3****************
0.8421052631578947
[1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0]
[1 0 1 0 1 0 1 1 1 1 1 0 1 0 1 0 1 0 0]
**********Fold 4****************
1.0
[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0]
[0 1 0 1 0 1 0 1 0 1 0 1 1 1 0 1 1 0 0]
**********Fold 5****************
1.0
[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0]
[0 1 0 1 0 1 0 1 0 1 1 1 0 1 0 1 0 1 0]
